# Nonlinear Koiter Shells via Hellan-Herrmann-Johnson (HHJ) method
The Lagrangian reads
\begin{align}
\mathcal{L}(u,\sigma) =\frac{t}{2}\|E_{\mathcal{S}}\|^2_{\mathbb{C}} -\frac{6}{t^3}\|\sigma\|^2_{\mathbb{C}^{-1}} +  \sum_{\mathrm{triangles}}\int_{T} (F_{\mathcal{S}}^\top\nabla_{\mathcal{S}}(\nu\circ\phi)-\nabla_{\mathcal{S}}\hat{\nu}):\sigma\,ds + \sum_{\mathrm{edges}}\langle\sphericalangle(\nu_L,\nu_R)-\sphericalangle(\hat{\nu_L},\hat{\nu_R}),\sigma_{\hat{\mu}\hat{\mu}}\rangle - \int_{\mathcal{S}}f\cdot u\,ds,
\end{align}

### Cantilever with bending moments
We consider a beam which is fixed at the left boundary and we will apply a moment at the right boundary such that the beam should roll up to a circle (Possion ratio $\nu=0$). We use loadsteps to increase the moments and apply Newton's method. As the bending moment would be incorporated strongly via $\sigma_{\hat\mu\hat\mu}$, which is tedious, we use the hybridized formulation such that we can include the force weakly directly in the formulation. 

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import MakeStructuredSurfaceMesh

thickness = 0.1
L = 12
W = 1
E, nu = 1.2e6, 0
moment = IfPos(x-L+1e-6, 1, 0)*50*pi/3

mapping = lambda x,y,z : (L*x, W*y,0)
mesh = MakeStructuredSurfaceMesh(quads=False, nx=10, ny=1, mapping=mapping)
Draw(mesh)

In [ ]:
order = 1 #try out: 1,2,3

fes1 = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fes2 = VectorH1(mesh, order=order, dirichletx_bbnd="left", dirichlety_bbnd="left|bottom", dirichletz_bbnd="left")
fes3 = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="left")
fes  = fes2*fes1*fes3
u,sigma,hyb = fes.TrialFunction()
# trace needed as we are on the surface
sigma, hyb = sigma.Trace(), hyb.Trace()


We define the outer normal vector $\hat\nu$, tangential vector $\hat t$ and the co-normal vector $\hat\mu = \hat\nu\times \hat t$ at the initial configuration.

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $P_{\mathcal{S}}$, $F_{\mathcal{S}}$, $C_{\mathcal{S}}$, and $E_{\mathcal{S}}$ are defined.

Finally, the deformed normal and tangential vectors are declared, which depend on the unknown displacement field $u$.

In [ ]:
nsurf = specialcf.normal(mesh.dim)
t     = specialcf.tangential(mesh.dim)
nel   = Cross(nsurf, t)
    
Ptau    = Id(mesh.dim) - OuterProduct(nsurf,nsurf)
Ftau    = grad(u).Trace() + Ptau
Ctau    = Ftau.trans*Ftau
Etautau = 0.5*(Ctau - Ptau)

nphys   = Normalize(Cof(Ftau)*nsurf)
tphys   = Normalize(Ftau*t)
nelphys = Cross(nphys,tphys)

gradn = specialcf.Weingarten(3) #Grad(nsurf)
Hn = CF( (u.Operator("hesseboundary").trans*nphys), dims=(3,3) )

In [ ]:
fesVF = VectorFacetSurface(mesh, order=order)
        
gfclamped = GridFunction(FacetSurface(mesh,order=0))
gfclamped.Set(1,definedon=mesh.BBoundaries("left"))

solution = GridFunction(fes, name="solution")
gfb = GridFunction(fesVF)
gfbR = GridFunction(fesVF)

gfgradu  = Grad(solution.components[0])
gfnphys = Normalize(Cof(Ptau+gfgradu)*nsurf)

cfn  = Normalize(CF( gfb.components ))
cfnR = Normalize(CF( gfbR.components ))

pnaverage = Normalize( cfn - (tphys*cfn)*tphys )

gfb.Set((1-gfclamped)*gfnphys+gfclamped*nsurf, definedon=mesh.Boundaries(".*"), dual=True)
gfbR.vec.data = gfb.vec

Define the material norms $\|\cdot\|_{\mathbb{C}}^2$ and $\|\cdot\|_{\mathbb{C}^{-1}}^2$
\begin{align}
\mathbb{C} E_{\mathcal{S}} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)E_{\mathcal{S}}+\bar \nu\,\mathrm{tr}(E_{\mathcal{S}})P_{\mathcal{S}}\big)
\end{align}

In [ ]:
def MaterialNorm(mat, E, nu):
    return E/(1-nu**2)*((1-nu)*InnerProduct(mat,mat)+nu*Trace(mat)**2)

def MaterialNormInv(mat, E, nu):
    return (1+nu)/E*(InnerProduct(mat,mat)-nu/(2*nu+1)*Trace(mat)**2)

Define the bilinear form for the problem including membrane and bending energy.
\begin{align}
\mathcal{L}(u,\sigma) =\frac{t}{2}\|E_{\mathcal{S}}\|^2_{\mathbb{C}} -\frac{6}{t^3}\|\sigma\|^2_{\mathbb{C}^{-1}} +  \int_{\mathcal{S}} \Big(\sum_{i=1}^3(\nabla_{\mathcal{S}}^2u_i)(\nu\circ\phi)_i+(1- \hat{\nu}\cdot\nu\circ\phi)\nabla_{\mathcal{S}}\hat{\nu}\Big):\sigma\,ds + \sum_{\mathrm{edges}}\langle\sphericalangle(\nu_L,\nu_R)-\sphericalangle(\hat{\nu_L},\hat{\nu_R}),\sigma_{\hat{\mu}\hat{\mu}}\rangle - \int_{\mathcal{S}}f\cdot u\,ds,
\end{align}

In [ ]:
bfA = BilinearForm(fes, symmetric=True, condense=True)
#membrane energy
bfA += Variation( 0.5*thickness*MaterialNorm(Etautau, E, nu)*ds )

#bending energy
bfA += Variation( (-6/thickness**3*MaterialNormInv(sigma, E, nu) \
                   + InnerProduct(Hn + (1-nsurf*nphys)*gradn,sigma))*ds ).Compile()
# boundary term of bending energy including hybridization variable
bfA += Variation( (acos(nelphys*pnaverage)-acos(nel*cfnR)+hyb*nel)*(sigma*nel*nel)*ds(element_boundary=True) ).Compile()

# moment as right-hand side
par = Parameter(0.0)
bfA += Variation( -par*moment*(hyb*nel)*ds(element_boundary=True) )

In [ ]:
solution.vec[:]=0
scene = Draw(solution.components[0], mesh, "disp", deformation=solution.components[0])

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The normal vector needs to be averaged after each Newton step.

In [ ]:
numsteps=20
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps+1)/numsteps)
        print("Loadstep =", steps+1, ", F/Fmax =", (steps+1)/numsteps*100, "%")
        
        # update averaged normal vector
        gfb.Set((1-gfclamped)*gfnphys + gfclamped*nsurf, definedon=mesh.Boundaries(".*"), dual=True)
        
        # solve
        solvers.Newton(bfA, solution, inverse="sparsecholesky", printing=False, maxerr=2e-10, maxit=20)
        scene.Redraw()